<div align="justify"><h1><b><font size="6">
  Input Assumptions for Modelling the Tertiary Sector in the negaWatt-BE Scenario (update)</h1></font></div>
<div align="justify"><h2><b><font size="5">
  DEMAND OF TERTIARY SECTOR IN BELGIUM
</h2></font></b></div>

---
<div align="justify"><h3><font size="3">
  <b> Developers:</b> LATERRE Antoine
</h3></font></div> 
---

To ensure transparency and consistency within the negaWatt-BE project, this notebook documents all key input assumptions related to the tertiary sector. It generates the numerical values and hypotheses presented in the negaWatt-BE Scenario (update). These are ultimately exported in .csv file to be used in our PyPSA model.

<font size="5"><b>Table of Contents</b></font><a class="anchor" id="contents"></a> <br>
1. [xxx](#section_1)
    - 1.1. [

<font size="5"><b>Definitions</b></font><a class="anchor" id="references"></a> <br>
* **Tertiary sector**:
* **Usefull energy demand**:
* **Final energy consumption (or end-use demand)**:
* 

<font size="5"><b>References</b></font><a class="anchor" id="references"></a> <br>

* [1] Bureau fédéral du Plan; SPF Economie - Statbel (2025). *Population projections 2024-2070.* Population de la Belgique par âge, au 1er janvier - FR (xlsx). Retrieved November 10, 2025. [Database].<br>
  Available at: https://www.plan.be/en/data/population-projections-2024-2070
* [2] Rozsai, Mate; Jaxa-Rozen, Marc; Salvucci, Raffaele; Sikora, Przemyslaw; Gea Bermudez, Juan; Neuwahl, Frederik (2025): JRC-IDEES-2023. European Commission, Joint Research Centre (JRC). [Dataset].<br>
  PID: http://data.europa.eu/89h/1f0b480c-6d21-4d95-897d-20c7ca33df6f
* [3] Rozsai, Mate; Jaxa-Rozen, Marc; Salvucci, Raffaele; Sikora, Przemyslaw; Tattini, Jacopo; Neuwahl, Frederik (2024): JRC-IDEES-2021: the Integrated Database of the European Energy System – Data update and technical documentation, Publications Office of the European Union, Luxembourg, 2024. [Report].<br>
  DOI: https://doi.org/10.2760/614599
* [4] Eurostat (2025). *Population on 1 January.* (TPS00001) Retrieved December 18, 2025. [Database].<br>
  DOI: https://doi.org/10.2908/TPS00001
* [5] Eurostat (2023). *Population on 1st January by age, sex and type of projection.* (PROJ_23NP) Retrieved December 18, 2025. [Database].<br>
  DOI: https://doi.org/10.2908/PROJ_23NP
* [6] Bureau fédéral du Plan; SPF Economie - Statbel (2025). *Perspectives de ménages 2024-2070.* Ménages privés Belgique par âge, au 1er janvier - FR (xlsx). Retrieved November 20, 2025. [Database].<br>
    Available at: https://www.plan.be/fr/donn%C3%A9es/perspectives-de-menages-2024-20 


<font size="5"><b>Project functions</b></font><a class="anchor" id="project_functions"></a>

In [1]:
# Automatically reload the file if it is modified:
%load_ext autoreload
%autoreload 2

# Import packages:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the macro parameters:
%run ./demand_model_macro_nW-BE-update.ipynb

# Option to print and plot the results:
post_process = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
<font size="5"><b>1. Method</b></font> <a class="anchor" id="section_1"></a>
---

Use JRC-IDEES-2023 as baseline for reference year (2019)!

- Top-down:
- Bottum-up approach, with top down calibration for base year 
- National/regional final energy consumption — service/tertiary sector (total by fuel and electricity). Source: Statbel / Directorate-General Energy energy statistics. 
- Building stock by sub-sector and region: floor area (m²) and number of buildings for categories like offices, retail, education, healthcare, hotels, restaurants, public administration, warehouses. Sources: regional statistics (Flanders/Wallonia/Brussels), Statbel, municipal databases, or ORBIS/company databases for large users. 
- End-use intensities: kWh/m²/yr or kWh/employee/yr for heating, cooling, DHW, cooking, lighting, appliances/IT. Use ODYSSEE-MURE country profiles, TABULA/EPB typologies, literature and national surveys. 
- Climate data: heating degree-days (HDD) and cooling degree-days (CDD) by region (to scale heating/cooling). Use local meteorological datasets.
- Technology & fuel splits: share of gas, district heat, electricity, heating oil, biomass in tertiary heating (from Statbel / Belgian studies). Recent Belgian analyses show distributed heat and electricity dominate service sector consumption; fuel mix has been shifting (oil down since 2011). 
- Occupancy & operational schedules: typical hours/week and seasonal factors for each sub-sector (schools, offices, shops differ a lot).
- Appliance/equipment stock: lighting W/m², server/IT loads, refrigeration in retail/hospitality. Use measured load factors when available.

- 

> **Comment**: La collecte données globales pour la Belgique est un challenge, car la compétence est régionale.


---
<font size="5"><b>2. Usefull Energy Demand</b></font> <a class="anchor" id="section_2"></a>
---


In [2]:
df_tertirary = pd.DataFrame(index=years)

<font size="4"><b>2.1. Total useful surface area</b></font> <a class="anchor" id="section_2_1"></a>

En Belgique, la surface utile totale dédiée au secteur des services passe de 175.124,336 Mm² en 2000 à 227.073,150 Mm² en 2019 selon les données du JRC [JRC-IDESS-2023]. La surface utile correspondante par habitant croît de manière logarithmique, passant ainsi de 17,104 m²/hab en 2000 à 19,864 m²/hab en 2019.

> **Comment**: Il y a un offsconstant de +17%Mm² entre les donn et JRC-IDESS-3 par rapport à JRC-IDESS-2015 (de 2000 à 2015)02

La version de 2015 propose une methodological note, mais ne précise pas comment cette surface a été obtenue.
Dans la note de la version de 2021, il est précisé que "The total useful surface area is aligned to the EU Building Observatory (BSO), which provides data for all Member States for 2020. The evolution of surface area in other years (and consequently the evolution of the stock of representative buildings) is then estimated from macro-economic data like population, GDP per capita, and number of employees."
La version de 2023 étend les valeurs de la version de 2021 aux années 2022-2023, mais ne propose pas (encore?) de note méthodologique.

La version de 2015 a été utilisée pour le scénario CLEVER et pour nW-BE 2025, supposant ainsi une surface totale de 187.878,192 Mm² en 2015 et 190.000,000 Mm² pour toutes les années suivantes (soit une diminution de la surface disponible par capita).

> **Comment**: Il serait pertinent de trouver d'autres sources de données pour comparer (e.g. ODYSSEE).
 
La base de donnée ODYSSEE ne fournit pas de données pour la Belgique. Néanmoins, elle en fournit pour les pays voisins (voir demand_data_aux.ipynb). Pour certains pays, il y a un match acceptable (e.g. NL, FR). Pour d'autres, il y a des différences significatives (e.g. DE, EU27). Les chiffres belges de JRC-IDEES-2023 étant proches de ceuxi des NL, ceuxi-ci sont pris pour acceptables.

$\textcolor{Orange}{\textbf{Projection Assumption (Sufficiency)}}$<br>
- Reduce by 5% the surface per capita (from 19.864 m²/person in 2019 to 18.871 m²/person in 2025).

In [3]:
# Input data for 2019 from [JRC-IDESS-2023]
base_tot_sur = 227073.149760375e+3 
base_spe_sur = base_tot_sur/population_dict.loc[2019]
# Loading data into database
df_tertirary["tertiary_floor_area_spe [m2/person]"] = linear_growth(start_year=2019, start_value=base_spe_sur, end_year=2050, end_value=0.95*base_spe_sur, target_years=years)
df_tertirary["tertiary_floor_area_tot [Mm2]"]       = population_dict * df_tertirary["tertiary_floor_area_spe [m2/person]"] * 1e-6
# Printing data
df_tertirary.round(3).T

,2019,2025,2030,2035,2040,2045,2050
tertiary_floor_area_spe [m2/person],19.864,19.672,19.512,19.351,19.191,19.031,18.871
tertiary_floor_area_tot [Mm2],227.073,232.446,234.610,235.825,236.955,237.710,237.792


<font size="4"><b>2.2. Space heating</b></font> <a class="anchor" id="section_2_2"></a>

En 2019, la demande énergétique utile totale pour chauffage des surfaces tertiaires s'élevait à 2.540,027 ktoe [JRC-IDEES-2023], soit 29.541 TWh. La demande spécif

In [4]:
# Input data for 2019 from [JRC-IDESS-2023]
space_heating_tot = 2540.02724091575*ktoe_to_GWh*1e-3 # TWh
space_heating_spe = space_heating_tot/base_tot_sur*1e+9
# Printing data
print(f'Total    usefull heating demand:  {round(space_heating_tot,3)} [TWh]')
print(f'Specific usefull heating demand: {round(space_heating_spe,3)} [kWh/m²]')

Total    usefull heating demand:  29.541 [TWh]
Specific usefull heating demand: 130.093 [kWh/m²]


$\textcolor{Orange}{\textbf{Projection Assumption (Efficiency)}}$<br>
- Renovation and new buildings in the mix will reduce specific heating needs by xx% (claimed by EU)

$\textcolor{Orange}{\textbf{Projection Assumption (Sufficiency)}}$<br>
- Reduce setpoint to 19°C. -xx%

In [5]:
# Loading data into database
df_tertirary["dem_spe_heating [kWh/m2]"] = linear_growth(start_year=2019, start_value=space_heating_spe, end_year=2050, end_value=0.50*space_heating_spe, target_years=years)
df_tertirary["dem_tot_heating [TWh]"] = df_tertirary["dem_spe_heating [kWh/m2]"]*1e-9*df_tertirary["tertiary_floor_area_tot [Mm2]"]*1e+6
# Printing data
df_tertirary.round(3).T

,2019,2025,2030,2035,2040,2045,2050
tertiary_floor_area_spe [m2/person],19.864,19.672,19.512,19.351,19.191,19.031,18.871
tertiary_floor_area_tot [Mm2],227.073,232.446,234.610,235.825,236.955,237.710,237.792
dem_spe_heating [kWh/m2],130.093,117.503,107.012,96.520,86.029,75.538,65.046
dem_tot_heating [TWh],29.541,27.313,25.106,22.762,20.385,17.956,15.467


<font size="4"><b>2.3. Space cooling</b></font> <a class="anchor" id="section_2_3"></a>

<font size="4"><b>2.4. Hot water</b></font> <a class="anchor" id="section_2_4"></a>

In [6]:
# Distinguish three types of buildings: old (unrenovated), ren (renovated), new (new building with good performances)
df_tertirary["tertiary_share_ren [%]"] = [15, 20, 25, 30, 35, 40, 45] # existing, renovated
df_tertirary["tertiary_share_new [%]"] = [3, 4, 5, 6, 7, 8, 9] # new-built
df_tertirary["tertiary_share_old [%]"] = 100-df_tertirary["tertiary_share_new [%]"]-df_tertirary["tertiary_share_ren [%]"] # existing, non-renovated


df_inputs = pd.concat([
    df_tertirary
], axis=1).round(2).T


# df_inputs
df_tertirary.round(3).T

,2019,2025,2030,2035,2040,2045,2050
tertiary_floor_area_spe [m2/person],19.864,19.672,19.512,19.351,19.191,19.031,18.871
tertiary_floor_area_tot [Mm2],227.073,232.446,234.610,235.825,236.955,237.710,237.792
dem_spe_heating [kWh/m2],130.093,117.503,107.012,96.520,86.029,75.538,65.046
dem_tot_heating [TWh],29.541,27.313,25.106,22.762,20.385,17.956,15.467
tertiary_share_ren [%],15.000,20.000,25.000,30.000,35.000,40.000,45.000
tertiary_share_new [%],3.000,4.000,5.000,6.000,7.000,8.000,9.000
tertiary_share_old [%],82.000,76.000,70.000,64.000,58.000,52.000,46.000


Total useful surface area

Comme les performances énergétiques des batiments anciens, rénovés et neufs ne sont pas le mêmes, il est utile d'effectuer une distinction entre ceux-ci.

2540,02724091575 ktoe

Le surface utile totale des batîments neufs et rénovés passe de (1995) à 3.216,441 Mm² (2019)

Pour pr
Pour évaluer la surface totale de batiments résidentiels et prédire la consommation

The average household size represents the average number of people living per dwelling. It is expressed in people per dwelling.

In [7]:
# SPACE HEATING
# -> kWh_th (PyPSA optimally select the space heating tech)
df_tertirary["spe_heating_demand_old [kWh/m2/year]"] = [250, 250, 250, 250, 250, 250, 250]
df_tertirary["spe_heating_demand_ren [kWh/m2/year]"] = [150, 150, 150, 150, 150, 150, 150]
df_tertirary["spe_heating_demand_new [kWh/m2/year]"] = [ 50,  50,  50,  50,  50,  50,  50]

# SPACE COOLING
# -> kWh_th + kWh_el (PyPSA does not model the space cooling tech)
df_tertirary["spe_heating_demand_old [kWh/m2/year]"] = [ 20,  20,  20,  20,  20,  20,  20]
df_tertirary["spe_heating_demand_ren [kWh/m2/year]"] = [150, 150, 150, 150, 150, 150, 150]
df_tertirary["spe_heating_demand_new [kWh/m2/year]"] = [ 50,  50,  50,  50,  50,  50,  50]

# SANITARY HOT WATER
# -> kWh_th (PyPSA optimally select the water heating tech)

# COOKING
# -> kWh_gas or kWh_elec

# OTHER ELECTRICAL APPLIANCES
# -> including refrieration